# Code for pressure drop after adjoint design optimization from .csv file obtained from Paraview

In [2]:
import os
import pandas as pd
from csv import reader  # for reading csv files
from csv import writer  # for writing csv files

    Sorting function

In [2]:
def sort(x,y):
    for i in range(len(x)):
         for j in range(i + 1, len(x)):
             if x[i] >x[j]:
                x[i], x[j] = x[j], x[i]
                y[i], y[j] = y[j], y[i]

    Computing pressure drop

In [3]:
def compute_pressure_drop(case):
    x_inlets = []
    x_outlets = []
    pressure_inlets =[]
    pressure_outlets =[]
    for folders in os.listdir('/home/cristopher/Desktop/Documentation/results_optimization/'+case+'/pressure'):
        x= str(folders)
        if (x.split("_")[1]=="inlet"):
            df=pd.read_csv('/home/cristopher/Desktop/Documentation/results_optimization/'+case+'/pressure/'+x,header=None).values #[1:,23]
            for i in range(0,df.shape[1]):
                if df[0,i]=='Surface_Total_Pressure_Inlet':
                    x_inlets.append(int(x.split("_")[2].split(".")[0]))
                    pressure_inlets.append(float(df[1,int(i)]))
        if (x.split("_")[1]=="outlet"):
            df=pd.read_csv('/home/cristopher/Desktop/Documentation/results_optimization/'+case+'/pressure/'+x,header=None).values #[1:,23]
            for i in range(0,df.shape[1]):
                if df[0,i]=='Surface_Total_Pressure_Outlet':
                    x_outlets.append(int(x.split("_")[2].split(".")[0]))
                    pressure_outlets.append(float(df[1,int(i)]))               
    sort(x_inlets,pressure_inlets)
    sort(x_outlets,pressure_outlets)
    pressure_drop =[]
    with open('/home/cristopher/Desktop/Documentation/results_optimization/'+case+'/pressure_drop.csv', 'w', newline='\n') as write_obj:
        writer(write_obj).writerow(['Iteration','Pressure Drop'])
        for i in range(0,len(pressure_outlets)):
            writer(write_obj).writerow([x_outlets[i],float(pressure_inlets[i]-pressure_outlets[i])])    

In [4]:
compute_pressure_drop('kenics_mixer_optimization_two_boxes_without_edges_double_points_45_4_constraint')

    Computing covariance

In [5]:
def compute_covariance(case):
    directory = '/home/cristopher/Desktop/Documentation/Simulations/SU2_TUTORIAL_TRANSPORT_PROPERTIES/'+case+'/optimization/'
    x_position =[]
    covariance = []
    for folders in os.listdir(directory):
        x = str(folders)
        if (x.split("_")[0]=="DSN"):
            df = pd.read_csv(directory+x+'/DIRECT/history.csv', header=None).values
            for i in range(0, df.shape[1]):
                if df[0,i]=='Avg_Species_0(outlet)':
                    mean = float(df[df.shape[0]-1,i])
                if df[0,i]=='Species_Variance(outlet)':
                    variance = float(df[df.shape[0]-1,i])
            covariance.append(float(variance/mean))
            x_position.append(int(x.split("_")[1]))
    sort(x_position,covariance)
    with open('/home/cristopher/Desktop/Documentation/results_optimization/'+case+'/COV.csv', 'w', newline='\n') as write_obj:
        writer(write_obj).writerow(['Iteration','COV'])
        for i in range(0,len(covariance)):
            writer(write_obj).writerow([x_position[i],float(covariance[i])]) 
    print(covariance)                   

In [1]:
compute_covariance('kenics_mixer_optimization_two_boxes_without_edges_double_points_45_4')